# da-instaseis: Getting Started

This notebook demonstrates how to use **da-instaseis** to:
1. Load an instaseis database
2. Extract synthetic Green's functions
3. Convolve with a source time function
4. Visualise the results with matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Install / import instaseis

Make sure you activated the pixi environment first (`pixi shell`).

In [ ]:
try:
    import instaseis
    print(f'instaseis version: {instaseis.__version__}')
except ImportError:
    print('instaseis not found – run `pixi install` in the repo root.')

## 2. Simulate a Gaussian source time function

In [ ]:
dt = 0.1            # seconds
duration = 20.0     # seconds
t = np.arange(0, duration, dt)

# Gaussian STF centred at 5 s with half-width 1 s
t0, sigma = 5.0, 1.0
stf = np.exp(-0.5 * ((t - t0) / sigma) ** 2)
stf /= stf.sum() * dt   # normalise to unit moment

fig, ax = plt.subplots(figsize=(8, 3))
ax.plot(t, stf)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.set_title('Gaussian source time function')
plt.tight_layout()
plt.show()

## 3. Synthetic convolution example

The cell below demonstrates `convolve_stf` using a mock stream
(no instaseis database required).

In [ ]:
from da_instaseis.waveforms import convolve_stf
from da_instaseis.plotting import plot_waveforms

# Build a toy obspy-like stream using obspy
import obspy

# Create a simple spike seismogram
data = np.zeros(len(t))
data[len(t) // 4] = 1.0

tr = obspy.Trace(data=data)
tr.stats.sampling_rate = 1.0 / dt
tr.stats.channel = 'BHZ'
st = obspy.Stream(traces=[tr])

# Convolve with the Gaussian STF
st_conv = convolve_stf(st, stf, dt)

fig, axes = plot_waveforms(st_conv, title='Spike convolved with Gaussian STF')
plt.show()

## 4. PyTorch – quick sanity check

In [ ]:
try:
    import torch
    x = torch.tensor([1.0, 2.0, 3.0])
    print(f'torch version : {torch.__version__}')
    print(f'CUDA available: {torch.cuda.is_available()}')
    print(f'tensor        : {x}')
except ImportError:
    print('torch not found – run `pixi install` in the repo root.')